# Clase 3: Acceso a Datos "No-ETL"

En el mundo moderno, a veces no hay tiempo para construir pipelines complejos de ingestión de datos (ETL). Necesitamos acceder a la data **donde vive** (archivos CSV, JSON, Logs) y limpiarla sobre la marcha.

## Objetivo de Aprendizaje
- Entender concepto "No-ETL".
- Cargar archivos de datos "sucios" (con errores típicos).
- Realizar limpieza básica usando Python y Pandas.

---

## Paso 1: Configurar Kaggle
Requerido para bajar el dataset base de Baseball, aunque hoy simularemos datos sucios también.

In [ ]:
from google.colab import files
import os

if not os.path.exists("/root/.kaggle/kaggle.json"):
    print("Por favor, sube tu archivo kaggle.json:")
    uploaded = files.upload()
    
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("Credenciales listas.")

In [ ]:
if not os.path.exists("baseball_data"):
    !kaggle datasets download -d open-source-sports/baseball-databank
    !unzip -q baseball-databank.zip -d baseball_data
    print("Dataset descargado.")

## Paso 2: Simulando Datos Sucios

En la vida real, los excels que nos mandan los entrenadores o scouts vienen con errores: celdas vacías, mayúsculas mezcladas con minúsculas, espacios extra, etc.

Vamos a crear un pequeño CSV "sucio" para practicar.

In [ ]:
import pandas as pd
import numpy as np

# Datos sucios simulados
data_sucia = {
    'Jugador': ['  Lionel Messi', 'Cristiano Ronaldo', 'Neymar JR', np.nan, 'Mbappe'],
    'Goles': ['30', '25', 'N/A', '15', '20'],  # ¡String con texto en vez de números!
    'Equipo': ['miami', 'Al Nassr', 'Al Hilal', 'Man City', 'PSG'] # Inconsistencia en mayúsculas
}

df_sucio = pd.DataFrame(data_sucia)
df_sucio.to_csv('datos_sucios.csv', index=False)

print("Archivo 'datos_sucios.csv' creado.")
# Leemos el archivo tal cual viene
df = pd.read_csv('datos_sucios.csv')
df

### Problemas visibles:
1.  **Espacios:** '  Lionel Messi' tiene espacios al inicio.
2.  **Valores Nulos:** Hay un `NaN` en Jugadores y un 'N/A' (texto) en Goles.
3.  **Tipos de Datos:** La columna Goles debería ser numérica, pero es objeto (texto) por culpa del 'N/A'.
4.  **Texto:** 'miami' está en minúsculas, otros en Título.

## Paso 3: Limpieza (Data Cleaning)

Vamos a arreglar esto paso a paso.

In [ ]:
# 1. Eliminar espacios en blanco en cadenas de texto
df['Jugador'] = df['Jugador'].str.strip()

# 2. Estandarizar texto de Equipos (Todo a Mayúsculas)
df['Equipo'] = df['Equipo'].str.upper()

# 3. Manejar Goles no numéricos
# Convertimos a numérico, forzando los errores (N/A) a NaN (Not a Number)
df['Goles'] = pd.to_numeric(df['Goles'], errors='coerce')

# 4. Rellenar o eliminar nulos
# Opción A: Eliminar fila si no tiene nombre de jugador
df = df.dropna(subset=['Jugador'])

# Opción B: Rellenar Goles vacíos con 0
df['Goles'] = df['Goles'].fillna(0)

df

## Paso 4: Práctica Real con Baseball Databank

Ahora carga el archivo `baseball_data/Salaries.csv` y verifica si hay datos nulos.

In [ ]:
try:
    df_salarios = pd.read_csv('baseball_data/Salaries.csv')
    print("Salarios cargados.")
    print(df_salarios.info())
    
    # Muestra cuantos nulos hay por columna
    print("\nValores nulos por columna:")
    print(df_salarios.isnull().sum())
except:
    print("Recuerda descargar el dataset primero.")